# KAN vs Multivariate Linear Regression: Sigman-Style Analysis

This notebook demonstrates using **Kolmogorov-Arnold Networks (KANs)** on a real dataset from the **Sigman group** that was originally analyzed with traditional statistical methods.

## Background: MLR in Physical Organic Chemistry

The [Sigman Lab](https://www.sigmanlab.com/) pioneered the use of **Multivariate Linear Regression (MLR)** with computed molecular descriptors to predict reaction outcomes. Key features of this approach:

1. **Sterimol Parameters**: Steric descriptors (B1, B5, L) that quantify molecular shape
2. **NBO Charges**: Electronic descriptors from Natural Bond Orbital analysis
3. **Buried Volume**: %V_bur measures how much space a substituent occupies
4. **NMR Shifts**: Computed chemical shifts as electronic probes

**The MLR Assumption**: These descriptors have **linear relationships** with reaction outcomes.

**But what if the relationships are nonlinear?** This is where KAN excels!

## Dataset: Thioetherification Reaction Success

We use data from [SigmanGroup/Thioetherification-modeling](https://github.com/SigmanGroup/Thioetherification-modeling):
- **153 reactions** with electrophile + nucleophile combinations
- **38 DFT-computed descriptors** per reaction
- **Target**: Reaction success (binary: works/doesn't work)

## References

- Santiago, C. B., Guo, J.-Y., & Sigman, M. S. (2018). [Predictive and mechanistic multivariate linear regression models for reaction development](https://pubs.rsc.org/en/content/articlehtml/2018/sc/c7sc04679k). *Chemical Science*.
- [morfeus](https://github.com/digital-chemistry-laboratory/morfeus) - Python package for molecular features
- [kraken](https://github.com/SigmanGroup/kraken) - Phosphine ligand discovery platform

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report, confusion_matrix
from sklearn.decomposition import PCA
import seaborn as sns

# KAN from pycse
import os
os.environ["JAX_PLATFORM_NAME"] = "cpu"
from pycse.sklearn.kan import KAN

import warnings
warnings.filterwarnings('ignore')

# Set style
plt.style.use('seaborn-v0_8-whitegrid')

## 1. Load the Sigman Group Dataset

In [ ]:
# Download the dataset from SigmanGroup GitHub
import urllib.request
import os

data_url = "https://github.com/SigmanGroup/Thioetherification-modeling/raw/main/notebooks/dataset.xlsx"
data_path = "/tmp/thioetherification_dataset.xlsx"

if not os.path.exists(data_path):
    print("Downloading Sigman group Thioetherification dataset...")
    urllib.request.urlretrieve(data_url, data_path)
    print("Done!")

# Load training/testing data
df_train = pd.read_excel(data_path, sheet_name='training_testing')
df_val = pd.read_excel(data_path, sheet_name='validation')

print(f"Training/Test set: {len(df_train)} reactions")
print(f"Validation set: {len(df_val)} reactions")
print(f"\nColumns: {len(df_train.columns)}")

In [ ]:
# Examine the descriptor types
desc_cols = [c for c in df_train.columns if c.startswith('e_') or c.startswith('n_')]

print("=" * 60)
print("DESCRIPTOR CATEGORIES (Sigman-style)")
print("=" * 60)

# Categorize descriptors
categories = {
    'Sterimol': [c for c in desc_cols if 'Sterimol' in c],
    'NBO Charge': [c for c in desc_cols if 'NBO' in c],
    'Buried Volume': [c for c in desc_cols if 'Vbur' in c],
    'NMR Shift': [c for c in desc_cols if 'NMR' in c],
    'LUMO': [c for c in desc_cols if 'LUMO' in c],
    'Dipole': [c for c in desc_cols if 'dipole' in c],
    'Volume/SASA': [c for c in desc_cols if 'volume' in c or 'SASA' in c],
    'Pyramidalization': [c for c in desc_cols if 'pyramidalization' in c],
    'Distance': [c for c in desc_cols if 'distance' in c],
}

for cat, cols in categories.items():
    if cols:
        print(f"\n{cat} ({len(cols)} descriptors):")
        for c in cols[:3]:
            print(f"  • {c}")
        if len(cols) > 3:
            print(f"  ... and {len(cols)-3} more")

In [ ]:
# Visualize the target distribution
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Target distribution
success_counts = df_train['Success'].value_counts().sort_index()
colors = ['#d62728', '#2ca02c']  # Red for failure, green for success
axes[0].bar(['Failure (0)', 'Success (1)'], success_counts.values, color=colors)
axes[0].set_ylabel('Number of Reactions')
axes[0].set_title('Reaction Outcome Distribution')
for i, v in enumerate(success_counts.values):
    axes[0].text(i, v + 2, f'{v} ({100*v/len(df_train):.1f}%)', ha='center', fontweight='bold')

# Descriptor counts by type
e_cols = [c for c in desc_cols if c.startswith('e_')]
n_cols = [c for c in desc_cols if c.startswith('n_')]
axes[1].bar(['Electrophile\nDescriptors', 'Nucleophile\nDescriptors'], 
            [len(e_cols), len(n_cols)], color=['#ff7f0e', '#1f77b4'])
axes[1].set_ylabel('Number of Features')
axes[1].set_title('DFT-Computed Descriptors')
for i, v in enumerate([len(e_cols), len(n_cols)]):
    axes[1].text(i, v + 0.5, str(v), ha='center', fontweight='bold')

plt.tight_layout()
plt.show()

print(f"\nTotal descriptors: {len(desc_cols)}")
print(f"Class balance: {100*success_counts[1]/len(df_train):.1f}% success rate")

## 2. Prepare Features for Modeling

The Sigman approach typically:
1. Standardizes all descriptors
2. Uses forward stepwise selection to find key descriptors
3. Builds an interpretable MLR model

We'll compare the full feature set with different models.

In [ ]:
# Prepare feature matrix
X = df_train[desc_cols].values
y = df_train['Success'].values

# Handle any NaN values
nan_mask = ~np.isnan(X).any(axis=1)
X = X[nan_mask]
y = y[nan_mask]

print(f"Feature matrix shape: {X.shape}")
print(f"Target shape: {y.shape}")
print(f"Class distribution: {np.bincount(y)}")

# Scale features (critical for MLR and KAN)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42, stratify=y
)

print(f"\nTraining set: {len(X_train)} samples")
print(f"Test set: {len(X_test)} samples")

## 3. Model Comparison: Traditional vs KAN

We compare:
1. **Logistic Regression** - Linear decision boundary (Sigman-style baseline)
2. **Ridge Classifier** - L2-regularized linear model
3. **Random Forest** - Nonlinear but less interpretable
4. **KAN** - Nonlinear AND interpretable!

In [ ]:
# Train and evaluate models
results = {}

# 1. Logistic Regression (MLR equivalent for classification)
print("Training Logistic Regression (linear baseline)...")
lr = LogisticRegression(max_iter=1000, random_state=42)
lr.fit(X_train, y_train)
y_pred_lr = lr.predict(X_test)
y_prob_lr = lr.predict_proba(X_test)[:, 1]
results['Logistic Regression'] = {
    'model': lr,
    'accuracy': accuracy_score(y_test, y_pred_lr),
    'auc': roc_auc_score(y_test, y_prob_lr),
    'y_pred': y_pred_lr,
    'y_prob': y_prob_lr
}
print(f"  Accuracy: {results['Logistic Regression']['accuracy']:.3f}")
print(f"  AUC: {results['Logistic Regression']['auc']:.3f}")

# 2. Random Forest
print("\nTraining Random Forest...")
rf = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)
y_prob_rf = rf.predict_proba(X_test)[:, 1]
results['Random Forest'] = {
    'model': rf,
    'accuracy': accuracy_score(y_test, y_pred_rf),
    'auc': roc_auc_score(y_test, y_prob_rf),
    'y_pred': y_pred_rf,
    'y_prob': y_prob_rf
}
print(f"  Accuracy: {results['Random Forest']['accuracy']:.3f}")
print(f"  AUC: {results['Random Forest']['auc']:.3f}")

In [ ]:
# 3. KAN - Nonlinear AND interpretable
print("Training KAN...")
n_features = X_train.shape[1]

# KAN for classification: output sigmoid probability
kan = KAN(
    layers=(n_features, 4, 1),  # Compact architecture
    grid_size=5,
    spline_order=3,
)

# Train on continuous labels (treat as regression, then threshold)
kan.fit(X_train, y_train.astype(float), maxiter=300)

# Predict probabilities (clipped to [0,1])
y_prob_kan = np.clip(kan.predict(X_test), 0, 1)
y_pred_kan = (y_prob_kan > 0.5).astype(int)

results['KAN'] = {
    'model': kan,
    'accuracy': accuracy_score(y_test, y_pred_kan),
    'auc': roc_auc_score(y_test, y_prob_kan),
    'y_pred': y_pred_kan,
    'y_prob': y_prob_kan
}
print(f"  Accuracy: {results['KAN']['accuracy']:.3f}")
print(f"  AUC: {results['KAN']['auc']:.3f}")

In [ ]:
# Summary comparison
print("=" * 60)
print("MODEL COMPARISON SUMMARY")
print("=" * 60)
print(f"{'Model':<25} {'Accuracy':<12} {'AUC':<12} {'Interpretable?'}")
print("-" * 60)

interpretability = {
    'Logistic Regression': 'Yes (coefficients)',
    'Random Forest': 'Partial (importance)',
    'KAN': 'Yes (activations)'
}

for name, res in results.items():
    print(f"{name:<25} {res['accuracy']:<12.3f} {res['auc']:<12.3f} {interpretability[name]}")

print("=" * 60)

In [ ]:
# Visualize predictions
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

for ax, (name, res) in zip(axes, results.items()):
    # Confusion matrix
    cm = confusion_matrix(y_test, res['y_pred'])
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax,
                xticklabels=['Failure', 'Success'],
                yticklabels=['Failure', 'Success'])
    ax.set_xlabel('Predicted')
    ax.set_ylabel('Actual')
    ax.set_title(f"{name}\nAccuracy: {res['accuracy']:.3f}, AUC: {res['auc']:.3f}")

plt.tight_layout()
plt.show()

## 4. Interpretability Comparison

### 4.1 Logistic Regression: Linear Coefficients

In MLR/Logistic Regression, coefficients show **linear effects** on the log-odds.

**Limitation**: Assumes each descriptor has a constant effect regardless of its value!

In [ ]:
# Logistic regression coefficients
lr_coefs = pd.DataFrame({
    'Feature': desc_cols,
    'Coefficient': lr.coef_[0]
}).sort_values('Coefficient', key=abs, ascending=False)

print("Top 10 Most Influential Features (Logistic Regression):")
print(lr_coefs.head(10).to_string(index=False))

# Visualize top features
top_n = 15
fig, ax = plt.subplots(figsize=(10, 6))
top_feats = lr_coefs.head(top_n)
colors = ['#2ca02c' if c > 0 else '#d62728' for c in top_feats['Coefficient']]
ax.barh(range(top_n), top_feats['Coefficient'], color=colors, alpha=0.7)
ax.set_yticks(range(top_n))
ax.set_yticklabels(top_feats['Feature'], fontsize=8)
ax.set_xlabel('Coefficient (log-odds per std. dev.)')
ax.set_title('Logistic Regression: Linear Effects on Reaction Success\n(Green = promotes success, Red = promotes failure)')
ax.axvline(0, color='black', linewidth=0.5)
ax.invert_yaxis()
plt.tight_layout()
plt.show()

### 4.2 Random Forest: Feature Importance

Random Forest tells us **which features matter** but not **how** they affect the outcome.

In [ ]:
# Random Forest feature importance
rf_importance = pd.DataFrame({
    'Feature': desc_cols,
    'Importance': rf.feature_importances_
}).sort_values('Importance', ascending=False)

print("Top 10 Most Important Features (Random Forest):")
print(rf_importance.head(10).to_string(index=False))

# Visualize
fig, ax = plt.subplots(figsize=(10, 6))
top_feats = rf_importance.head(top_n)
ax.barh(range(top_n), top_feats['Importance'], color='steelblue', alpha=0.7)
ax.set_yticks(range(top_n))
ax.set_yticklabels(top_feats['Feature'], fontsize=8)
ax.set_xlabel('Feature Importance')
ax.set_title('Random Forest: Feature Importance\n(No information about direction or shape of effect!)')
ax.invert_yaxis()
plt.tight_layout()
plt.show()

### 4.3 KAN: Learned Activation Functions

**This is the key advantage of KAN!**

Each input edge learns a **univariate activation function** that shows:
- The **shape** of the relationship (linear, sigmoidal, threshold, etc.)
- **Optimal ranges** for the descriptor
- **Nonlinear effects** that MLR cannot capture

In [ ]:
# Visualize KAN activations
print("KAN Input Layer Activations")
print("Each plot shows how a descriptor is transformed before contributing to prediction.")
kan.plot_activations(layer_idx=0, figsize=(16, 14))
plt.suptitle('KAN Learned Activations: Nonlinear Structure-Activity Relationships', y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
# Network visualization
kan.plot_network(figsize=(14, 10))
plt.title('KAN Network Architecture with Learned Activations')
plt.show()

## 5. Deep Dive: Comparing Linear vs Nonlinear Effects

Let's compare how logistic regression (linear) and KAN (nonlinear) model the effect of specific descriptors.

In [ ]:
# Select top features to compare
# Use features that are important in both models
lr_top = set(lr_coefs.head(10)['Feature'].tolist())
rf_top = set(rf_importance.head(10)['Feature'].tolist())
common_top = list(lr_top.intersection(rf_top))

if len(common_top) < 4:
    # If not enough overlap, use RF top features
    compare_features = rf_importance.head(6)['Feature'].tolist()
else:
    compare_features = common_top[:6]

print("Features to compare:")
for f in compare_features:
    print(f"  • {f}")

In [ ]:
def plot_partial_effect(model, feature_idx, X, feature_name, n_points=100, is_kan=False):
    """Compute partial dependence for a feature."""
    x_range = np.linspace(X[:, feature_idx].min(), X[:, feature_idx].max(), n_points)
    X_temp = np.tile(X.mean(axis=0), (n_points, 1))
    X_temp[:, feature_idx] = x_range
    
    if is_kan:
        y_pred = np.clip(model.predict(X_temp), 0, 1)
    else:
        y_pred = model.predict_proba(X_temp)[:, 1]
    
    return x_range, y_pred

# Plot partial effects
n_compare = min(6, len(compare_features))
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for ax, feat in zip(axes[:n_compare], compare_features[:n_compare]):
    idx = desc_cols.index(feat)
    
    # Logistic Regression (linear)
    x_range_lr, y_pred_lr = plot_partial_effect(lr, idx, X_scaled, feat)
    
    # KAN (nonlinear)
    x_range_kan, y_pred_kan = plot_partial_effect(kan, idx, X_scaled, feat, is_kan=True)
    
    ax.plot(x_range_lr, y_pred_lr, 'b--', linewidth=2, label='Logistic Reg (linear)')
    ax.plot(x_range_kan, y_pred_kan, 'r-', linewidth=2, label='KAN (nonlinear)')
    
    # Add actual data points
    for label, color, marker in [(0, '#d62728', 'x'), (1, '#2ca02c', 'o')]:
        mask = y == label
        ax.scatter(X_scaled[mask, idx], 
                  np.full(mask.sum(), label), 
                  alpha=0.3, s=20, c=color, marker=marker)
    
    ax.set_xlabel(f'{feat}\n(standardized)', fontsize=8)
    ax.set_ylabel('P(Success)')
    ax.set_ylim(-0.1, 1.1)
    ax.legend(fontsize=8)
    ax.grid(True, alpha=0.3)

# Hide unused subplots
for ax in axes[n_compare:]:
    ax.set_visible(False)

plt.suptitle('Partial Dependence: Linear vs Nonlinear Effects\n'
             'KAN can capture thresholds, optima, and saturation that MLR misses!', 
             fontsize=12)
plt.tight_layout()
plt.show()

## 6. Chemical Interpretation of KAN Activations

The learned activation functions can reveal chemical insights:

### Possible Nonlinear Patterns:

1. **Threshold Effects**: Descriptor must exceed a critical value
   - E.g., Sterimol B5 > threshold for steric protection

2. **Optimal Range**: Too little OR too much is bad
   - E.g., NBO charge has a "Goldilocks" zone

3. **Saturation**: Effect plateaus above certain value
   - E.g., Buried volume effect saturates at high coverage

4. **Synergistic Effects**: Combined through hidden nodes
   - Multiple descriptors interact nonlinearly

In [ ]:
# Analyze which features have the most nonlinear effects
print("=" * 60)
print("ANALYZING NONLINEARITY IN LEARNED ACTIVATIONS")
print("=" * 60)

# For each feature, compare KAN vs linear prediction variance
nonlinearity_scores = []

for idx, feat in enumerate(desc_cols):
    x_range = np.linspace(X_scaled[:, idx].min(), X_scaled[:, idx].max(), 50)
    X_temp = np.tile(X_scaled.mean(axis=0), (50, 1))
    X_temp[:, idx] = x_range
    
    # KAN predictions
    y_kan = np.clip(kan.predict(X_temp), 0, 1)
    
    # Linear fit to KAN predictions
    linear_fit = np.polyfit(x_range, y_kan, 1)
    y_linear = np.polyval(linear_fit, x_range)
    
    # Nonlinearity = residual from linear fit
    nonlinearity = np.sqrt(np.mean((y_kan - y_linear)**2))
    nonlinearity_scores.append((feat, nonlinearity))

# Sort by nonlinearity
nonlinearity_df = pd.DataFrame(nonlinearity_scores, columns=['Feature', 'Nonlinearity'])
nonlinearity_df = nonlinearity_df.sort_values('Nonlinearity', ascending=False)

print("\nMost Nonlinear Relationships (where MLR fails most):")
print(nonlinearity_df.head(10).to_string(index=False))

print("\nMost Linear Relationships (MLR works well):")
print(nonlinearity_df.tail(5).to_string(index=False))

In [ ]:
# Visualize the most nonlinear features
most_nonlinear = nonlinearity_df.head(4)['Feature'].tolist()

fig, axes = plt.subplots(2, 2, figsize=(12, 10))
axes = axes.flatten()

for ax, feat in zip(axes, most_nonlinear):
    idx = desc_cols.index(feat)
    
    x_range = np.linspace(X_scaled[:, idx].min(), X_scaled[:, idx].max(), 100)
    X_temp = np.tile(X_scaled.mean(axis=0), (100, 1))
    X_temp[:, idx] = x_range
    
    y_kan = np.clip(kan.predict(X_temp), 0, 1)
    y_lr = lr.predict_proba(X_temp)[:, 1]
    
    ax.plot(x_range, y_lr, 'b--', linewidth=2, label='Logistic Reg (linear assumption)')
    ax.plot(x_range, y_kan, 'r-', linewidth=3, label='KAN (learned nonlinearity)')
    ax.fill_between(x_range, y_lr, y_kan, alpha=0.2, color='orange', label='Nonlinearity gap')
    
    ax.set_xlabel(feat, fontsize=9)
    ax.set_ylabel('P(Success)')
    ax.set_title(f'Nonlinearity Score: {nonlinearity_df[nonlinearity_df["Feature"]==feat]["Nonlinearity"].values[0]:.4f}')
    ax.legend(fontsize=8)
    ax.grid(True, alpha=0.3)
    ax.set_ylim(-0.1, 1.1)

plt.suptitle('Features with Strongest Nonlinear Effects\n'
             'Orange shading = where linear assumption fails', fontsize=12)
plt.tight_layout()
plt.show()

## 7. Practical Recommendations

Based on this analysis, here's when to use each approach:

| Scenario | Recommended Model | Reason |
|----------|-------------------|--------|
| Quick initial analysis | Logistic Regression | Fast, interpretable baseline |
| Maximum predictive accuracy | Random Forest / XGBoost | Captures complex interactions |
| **Understanding nonlinear SAR** | **KAN** | **Shows shape of relationships** |
| Feature selection for MLR | KAN + RF | Identify truly linear features |
| Publication-ready mechanism | KAN | Interpretable nonlinear insights |

### KAN Workflow for Chemistry:

1. **Start with MLR** to establish linear baseline
2. **Train KAN** to capture nonlinearities
3. **Compare partial effects** to identify where linearity fails
4. **Interpret activations** for chemical insight
5. **Focus experiments** on nonlinear descriptor regions

In [ ]:
# Final summary
print("=" * 60)
print("FINAL SUMMARY")
print("=" * 60)

print("\nDataset: Sigman Group Thioetherification Modeling")
print(f"  Reactions: {len(df_train)}")
print(f"  Descriptors: {len(desc_cols)} (DFT-computed)")
print(f"  Task: Predict reaction success/failure")

print("\nModel Performance:")
for name, res in results.items():
    print(f"  {name}: Accuracy={res['accuracy']:.3f}, AUC={res['auc']:.3f}")

print("\nKey Insights:")
print("  1. KAN achieves competitive accuracy with Random Forest")
print("  2. Unlike RF, KAN shows HOW each descriptor affects outcome")
print("  3. Several descriptors show nonlinear effects that MLR misses")
print(f"  4. Most nonlinear feature: {nonlinearity_df.iloc[0]['Feature']}")

print("\nFor Chemistry Applications:")
print("  • Use KAN when you need both accuracy AND interpretability")
print("  • Examine activations to find optimal descriptor ranges")
print("  • Identify where linear models (MLR) assumptions fail")
print("=" * 60)